# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [4]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=29)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [5]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [6]:
bootstrap_idx(X_train)

array([ 86,  40,  56,  14, 110, 103,  30,  49,  88, 100,  44,  18,  93,
        78, 101,  61,  17,  98,  39,  48,  60,  92, 109,  40,  90,  68,
         7,  95,  23,  62,  93,  92,  21,  41,  86,  85,  13,  27,  41,
        63,  24,  83,  79,  29,  63,  33, 103,  87, 106, 111,  74,  20,
        88,  73,  84,  50,  15,  52,  81,  27,  32, 102, 107,  65,  56,
        57,  80,  97,  54,  38, 102,  89,  58,  77,  69,  15,  71,  13,
        66,  58,  16,  25,  16,  36,  13,   3,  56,  38,  39,  21,  43,
         1,  23,   6,  10,  58,  29,  17,  83,   7,  92,  18, 109,  81,
        17,  60,  84,  20,   8,  45,  82,  10])

In [7]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({np.int64(66): 4,
         np.int64(75): 4,
         np.int64(47): 3,
         np.int64(13): 3,
         np.int64(4): 3,
         np.int64(26): 3,
         np.int64(41): 3,
         np.int64(46): 3,
         np.int64(58): 3,
         np.int64(89): 3,
         np.int64(3): 2,
         np.int64(63): 2,
         np.int64(102): 2,
         np.int64(59): 2,
         np.int64(94): 2,
         np.int64(53): 2,
         np.int64(70): 2,
         np.int64(61): 2,
         np.int64(55): 2,
         np.int64(103): 2,
         np.int64(39): 2,
         np.int64(54): 2,
         np.int64(79): 2,
         np.int64(109): 2,
         np.int64(18): 2,
         np.int64(29): 2,
         np.int64(71): 2,
         np.int64(87): 2,
         np.int64(95): 2,
         np.int64(37): 2,
         np.int64(80): 1,
         np.int64(111): 1,
         np.int64(81): 1,
         np.int64(21): 1,
         np.int64(106): 1,
         np.int64(32): 1,
         np.int64(2): 1,
         np.int64(110): 1,
         

In [8]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [9]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [10]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({np.int64(0): 38, np.int64(1): 37, np.int64(2): 37})
Classes distribution in the bootstrap: Counter({np.int64(0): 43, np.int64(2): 41, np.int64(1): 28})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [14]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
list_of_classifiers = []

trees = [DecisionTreeClassifier(max_depth=4, random_state=0) for _ in range(5)]
[tree.fit(X_train, y_train) for tree in trees]
y_preds = pd.DataFrame([tree.predict(X_test) for tree in trees])
y_pred = y_preds.mode().values[0]
(y_pred == y_test).mean()


np.float64(0.9473684210526315)

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [16]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(estimator=DecisionTreeClassifier())
clf.fit(X_train, y_train).score(X_test, y_test)

0.9473684210526315

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [17]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import math
list_of_classifiers = []

trees = [DecisionTreeClassifier(max_features= int (math.sqrt(X_train.shape[1]))) for _ in range(5)]
[tree.fit(X_train, y_train) for tree in trees]
y_preds = pd.DataFrame([tree.predict(X_test) for tree in trees])
y_pred = y_preds.mode().values[0]
(y_pred == y_test).mean()

np.float64(0.9473684210526315)

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [18]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test)

0.9473684210526315

In [ ]:
from figures import plot_forest_interactive
plot_forest_interactive()